# Use Customized Notebook Template (CLAIMS_RESERVE_Template)

In [1]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#Modeling Libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier

from xgboost import XGBClassifier
#from catboost import CatBoostClassifier

from joblib import dump, load
import requests

# Initialize Snowflake Session using Default connection

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
table_name = 'AUTO_INSURANCE_CLAIMS_DATA'

In [4]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [5]:
df = sf_df.to_pandas()

In [6]:
df.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,POLICE_REPORT_AVAILABLE,TOTAL_CLAIM_AMOUNT_PAID,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,POLICY_BIND_DATE_CUSTOM
0,139,37,457069,17-08-2011,CT,250/500,1351,1589,16139812,609322,...,No Police Report Available,48182,538,20189,27455,Dodge,Neon,2005,No Fraud Reported,2011-08-17
1,292,52,429086,27-09-2010,CT,100/300,2216,2669,15406514,438830,...,Police Report Available,64579,11170,5093,48316,Audi,A3,1999,No Fraud Reported,2010-09-27
2,45,20,457562,13-01-2002,CT,100/300,929,1290,5629268,616164,...,No Police Report Available,32420,538,845,31037,Mercedes,E400,2014,No Fraud Reported,2002-01-13
3,49,67,471692,07-02-2012,CT,500/1000,699,970,18501194,445120,...,No Police Report Available,53780,7280,898,45602,Suburu,Legacy,2005,No Fraud Reported,2012-02-07
4,128,45,463009,18-06-2008,CT,100/300,1122,1384,16890664,442936,...,Police Report Available,94511,412,14545,79554,Ford,Escape,2013,No Fraud Reported,2008-06-18


In [7]:
#insurance_claim =  df.copy()
insurance_claim =  df[::1]

In [8]:
insurance_claim = insurance_claim[['MONTHS_AS_CUSTOMER', 'CUSTOMER_AGE', 'POLICY_NUMBER',
       'POLICY_BIND_DATE', 'POLICY_STATE', 'POLICY_CSL', 'POLICY_DEDUCTABLE',
       'POLICY_ANNUAL_PREMIUM', 'UMBRELLA_LIMIT', 'INSURED_ZIP', 'INSURED_SEX',
       'INSURED_EDUCATION_LEVEL', 'INSURED_OCCUPATION', 'INSURED_HOBBIES',
       'INSURED_RELATIONSHIP', 'CAPITAL_GAINS', 'CAPITAL_LOSS',
       'INCIDENT_DATE', 'INCIDENT_TYPE', 'COLLISION_TYPE', 'INCIDENT_SEVERITY',
       'AUTHORITIES_CONTACTED', 'INCIDENT_STATE', 'INCIDENT_CITY',
       'INCIDENT_LOCATION', 'INCIDENT_HOUR_OF_THE_DAY', 'INCIDENT_TIME_OF_DAY',
       'NUMBER_OF_VEHICLES_INVOLVED', 'PROPERTY_DAMAGE', 'BODILY_INJURIES',
       'WITNESSES', 'POLICE_REPORT_AVAILABLE', 'TOTAL_CLAIM_AMOUNT_PAID',
       'INJURY_CLAIM', 'PROPERTY_CLAIM', 'VEHICLE_CLAIM', 'AUTO_MAKE',
       'AUTO_MODEL', 'AUTO_YEAR', 'FRAUD_REPORTED']]

In [9]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [10]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [11]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [12]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [13]:
df = insurance_claim[::1]

In [475]:
#df_copy = insurance_claim.copy()

In [14]:
#actual_inference = df.copy()
actual_inference = df[::1]
actual_inference.drop('fraud_reported', axis = 1, inplace=True)

In [15]:
actual_inference.head()

,months_as_customer,customer_age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,139,37,457069,17-08-2011,CT,250/500,1351,1589,16139812,609322,...,0,3,No Police Report Available,48182,538,20189,27455,Dodge,Neon,2005
1,292,52,429086,27-09-2010,CT,100/300,2216,2669,15406514,438830,...,1,2,Police Report Available,64579,11170,5093,48316,Audi,A3,1999
2,45,20,457562,13-01-2002,CT,100/300,929,1290,5629268,616164,...,0,2,No Police Report Available,32420,538,845,31037,Mercedes,E400,2014
3,49,67,471692,07-02-2012,CT,500/1000,699,970,18501194,445120,...,2,1,No Police Report Available,53780,7280,898,45602,Suburu,Legacy,2005
4,128,45,463009,18-06-2008,CT,100/300,1122,1384,16890664,442936,...,0,1,Police Report Available,94511,412,14545,79554,Ford,Escape,2013


In [16]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [17]:
df['incident_hour_of_the_day'] = df['incident_hour_of_the_day'].astype(str).astype(int)

In [18]:
df['incident_hour_of_the_day'].dtype

dtype('int64')

In [19]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [20]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [21]:
cat_df.columns

Index(['policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [22]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

policy_csl: 
['250/500' '100/300' '500/1000']

insured_sex: 
['MALE' 'FEMALE']

insured_education_level: 
['High School' 'MD' 'Masters' 'Associate' 'College' 'PhD' 'JD']

insured_occupation: 
['handlers-cleaners' 'armed-forces' 'prof-specialty' 'sales'
 'machine-op-inspct' 'priv-house-serv' 'adm-clerical' 'exec-managerial'
 'tech-support' 'farming-fishing' 'craft-repair' 'transport-moving'
 'protective-serv' 'other-service']

insured_relationship: 
['wife' 'unmarried' 'other-relative' 'husband' 'own-child' 'not-in-family']

incident_type: 
['Single Vehicle Collision' 'Parked Car' 'Multi-vehicle Collision'
 'Vehicle Theft']

collision_type: 
['Front Collision' 'Rear Collision' 'Side Collision'
 'Details not Available']

incident_severity: 
['Minor Damage' 'Major Damage' 'Total Loss']

authorities_contacted: 
['Ambulance' 'Other' 'None' 'Police' 'Fire']

incident_time_of_day: 
['Morning to Noon' 'Night Time' 'Afternoon Hours' 'Early Morning Hours']

property_damage: 
['Property Damage' '

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232025 entries, 0 to 232024
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   months_as_customer           232025 non-null  int16 
 1   customer_age                 232025 non-null  int8  
 2   policy_csl                   232025 non-null  object
 3   policy_deductable            232025 non-null  int16 
 4   policy_annual_premium        232025 non-null  int16 
 5   umbrella_limit               232025 non-null  int32 
 6   insured_sex                  232025 non-null  object
 7   insured_education_level      232025 non-null  object
 8   insured_occupation           232025 non-null  object
 9   insured_relationship         232025 non-null  object
 10  capital_gains                232025 non-null  int32 
 11  capital_loss                 232025 non-null  int32 
 12  incident_type                232025 non-null  object
 13  collision_type

In [24]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [25]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64','int64'])

In [26]:
num_df.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')

In [27]:
cat_df.columns

Index(['policy_csl_250/500', 'policy_csl_500/1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed-forces',
       'insured_occupation_craft-repair', 'insured_occupation_exec-managerial',
       'insured_occupation_farming-fishing',
       'insured_occupation_handlers-cleaners',
       'insured_occupation_machine-op-inspct',
       'insured_occupation_other-service',
       'insured_occupation_priv-house-serv',
       'insured_occupation_prof-specialty',
       'insured_occupation_protective-serv', 'insured_occupation_sales',
       'insured_occupation_tech-support',
       'insured_occupation_transport-moving',
       'insured_relationship_not-in-family',
       'insured_relationship_other-relative', 'insured_relationship_own-child',
       'insured_relationship

In [28]:
# combining the Numerical and Categorical dataframes to get the final dataset
X = pd.concat([num_df, cat_df], axis = 1)

In [29]:
X.columns = X.columns.str.replace('/', '_')
X.columns = X.columns.str.replace(' ', '_')
X.columns = X.columns.str.replace('-', '_')

In [30]:
# splitting data into training set and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [31]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [32]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [33]:
# Scaling the numeric values in the dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [34]:
scaled_data_test = scaler.transform(num_df_test)

In [35]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
48058,0.879981,-1.287351,-1.758751,-1.447126,-1.068409,1.019758,2.173703,-1.217866,-0.449642,-1.023098,0.611741,-1.212282
175951,0.298649,1.880533,-0.193247,-1.489260,-0.608735,0.678298,-0.678812,0.005575,0.446634,-0.629869,-0.081834,-1.494458
71860,-0.842079,0.146140,1.037382,-0.785359,0.468743,-1.199734,-0.678812,1.229016,1.342910,0.825916,-0.943270,0.701528
94335,-1.171136,-1.190165,-0.006374,-0.822010,1.141898,1.190488,-0.678812,0.005575,-0.449642,-0.847054,-0.967559,0.414887
202589,-1.072419,-1.677964,-1.758751,0.647333,-1.718083,0.336837,0.747445,0.005575,-0.449642,0.603273,1.663802,0.949580


In [36]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
172527,-0.216873,0.396580,0.807588,-1.115286,-0.577708,0.166107,-0.678812,0.005575,1.342910,0.252726,-0.959166,1.089456
214774,1.066446,-0.803291,0.820448,-1.003170,-1.402879,-0.858274,-0.678812,-1.217866,0.446634,-1.007005,-0.612887,-0.091423
188855,-1.061450,-1.330337,-1.758751,1.481098,0.369127,0.849028,0.747445,-1.217866,0.446634,-1.031354,0.427474,-0.849484
27393,0.507051,-0.545375,-1.758751,0.305213,-1.139443,-0.175353,2.173703,-1.217866,1.342910,-1.030794,0.632088,-0.298144
129553,1.143226,-0.736009,-0.662075,-1.031632,-0.849575,0.849028,-0.678812,0.005575,-1.345918,1.858665,0.324595,1.295092


In [37]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [38]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [39]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [40]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Decision Tree

In [41]:
X_train.columns = X_train.columns.str.upper()
X_test.columns = X_test.columns.str.upper()

In [42]:
from sklearn import preprocessing 
# label_encoder object knows  
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
y_train_lb= label_encoder.fit_transform(y_train)

y_test_lb= label_encoder.transform(y_test) 

In [43]:
y_train_lb  = pd.Series(y_train_lb)
y_test_lb  = pd.Series(y_test_lb)

In [44]:
#from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train_lb)

DecisionTreeClassifier()

In [45]:
y_pred = dtc.predict(X_test)

In [46]:
y_pred

array([1, 1, 1, ..., 1, 0, 1])

In [47]:
y_pred_lb = label_encoder.inverse_transform(y_pred)

In [48]:
y_pred_lb

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [49]:
y_train_out = dtc.predict(X_train)
y_train_out_lb = label_encoder.inverse_transform(y_train_out)

In [50]:
y_train_out_lb

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [51]:
y_pred_lb

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [52]:
dtc_train_acc = accuracy_score(y_train_lb, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test_lb, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test_lb, y_pred))
print(classification_report(y_test_lb, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.99501784267416
[[ 8583   157]
 [  132 49135]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      8740
           1       1.00      1.00      1.00     49267

    accuracy                           1.00     58007
   macro avg       0.99      0.99      0.99     58007
weighted avg       1.00      1.00      1.00     58007



In [53]:
from fosforml import register_model

In [54]:
type(y_pred)

numpy.ndarray

In [55]:
y_pred =  pd.Series(y_pred)

In [56]:
type(X_train), type(X_test),type(y_train_lb),type(y_test_lb), type(y_pred), 

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [519]:
#y_pred_series.name = 'fraud_predicted'

In [57]:
from snowflake.ml.registry import Registry

model_registry = Registry(session=my_session,
                          database_name='FDC_Insurance',
                          schema_name='INS_CLAIMS_RESERV_SCHEMA'
                         )

In [59]:
model_registry.log_model(dtc,
                         model_name="Claims_Fraud_DTree_Classifier",
                         version_name="v3",
                         comment="Insurance claims model",
                         conda_dependencies=["scikit-learn==1.3.2"],
                         metrics=[],
                         sample_input_data= X_train,
                         python_version='3.9')

In [60]:
y_pred.name = "FRAUD_PREDICTED"

In [61]:
y_train_lb.name = 'FRAUD_REPORTED'
y_test_lb.name = 'FRAUD_REPORTED'

In [62]:
y_test_lb

0        1
1        1
2        1
3        1
4        1
        ..
58002    1
58003    1
58004    1
58005    0
58006    1
Name: FRAUD_REPORTED, Length: 58007, dtype: int64

In [63]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=dtc, 
    session=my_session,
    x_train=X_train,
    y_train=y_train_lb,
    x_test=X_test,
    y_test=y_test_lb,
    y_pred=y_pred,
    source="Notebook",
    dataset_name="AUTO_INSURANCE_CLAIMS_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Decision_Tree_Claim_Classifier",
    description="Decision tree model trained via Notebook to identify fraud claim application",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%


Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_0309078C_6F8E_4796_BF18_3789B23539DF_FDC_DECISION_TREE_CLAIM_CLASSIFIER' registered successfully."

### Push X_test to Snowflake for Model Deployment Setup

In [529]:
sf_df = my_session.createDataFrame(X_test)
sf_df.write.mode("overwrite").save_as_table("AUTO_INSURANCE_CLAIMS_DATA_XTEST")
my_session.table("AUTO_INSURANCE_CLAIMS_DATA_XTEST").show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Version 2 of Decision Tree Fraud Classifier Model

In [64]:
dtc = DecisionTreeClassifier(random_state=7)
dtc.fit(X_train, y_train_lb)

DecisionTreeClassifier(random_state=7)

In [65]:
y_pred = dtc.predict(X_test)
y_prob = dtc.predict_proba(X_test)[:,1]

In [66]:
dtc_train_acc = accuracy_score(y_train_lb, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test_lb, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test_lb, y_pred))
print(classification_report(y_test_lb, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9948626889858121
[[ 8575   165]
 [  133 49134]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      8740
           1       1.00      1.00      1.00     49267

    accuracy                           0.99     58007
   macro avg       0.99      0.99      0.99     58007
weighted avg       0.99      0.99      0.99     58007



In [67]:
y_pred = pd.Series(y_pred)
y_pred.name = "FRAUD_PREDICTED"

In [544]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=dtc, 
    session=my_session,
    x_train=X_train,
    y_train=y_train_lb,
    x_test=X_test,
    y_test=y_test_lb,
    y_pred=y_pred,
    source="Notebook",
    dataset_name="AUTO_INSURANCE_CLAIMS_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Decision_Tree_Claim_Classifier",
    description="Decision tree model trained via Notebook to identify fraud claim application",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%


Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_4CAC9B7E_5794_4BA1_8E11_719ABA32ECA5_FDC_CLAIMS_FRAUD_DTREE_CLASSIFIER' registered successfully."